<a href="https://colab.research.google.com/github/Diego251Fagundes/Data-Science-Machine-Learning-Studies/blob/main/notebook_encoding_didatico_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🧩 Notebook Didático — *Encoding* de Variáveis Categóricas (conceitos + exemplos claros)

**Enunciado (leia antes de começar):**  
Este notebook foi feito para **ensinar** (não apenas mostrar código). Você vai:
1) Entender **por que** precisamos codificar variáveis categóricas;  
2) Ver **quatro encodings** comuns (One-Hot, Ordinal, Frequência, Target Mean) com exemplos simples;  
3) Interpretar os **resultados transformados** (tabelas e nomes de colunas);  
4) Responder **perguntas curtas** ao final de cada seção para checar sua compreensão.

> **Importante:** aqui **não treinamos modelos**; o foco é **compreensão conceitual** e **transformações**.  
> Pré‑requisitos: noções básicas de `pandas` e `scikit-learn`.



## 🎯 Objetivos de Aprendizagem
Ao concluir este notebook, você será capaz de:
- Explicar, em suas palavras, o que é *encoding* e por que ele é necessário em ML/DS;  
- Aplicar **One-Hot** e **Ordinal** com `scikit-learn` e **interpretar** o resultado;  
- Implementar **Frequency** e **Target Mean** manualmente com `pandas` em exemplos controlados;  
- Identificar **riscos** como **ordem artificial** (Ordinal) e **data leakage** (Target Mean).



## 🔎 Nosso *dataset* didático
Vamos usar um conjunto **fabricado** de poucas linhas para visualizar claramente as transformações.  
Ele representa produtos de vestuário com três atributos **categóricos**:
- **`cor`**: `azul`, `verde`, `vermelho` (nominal, **sem** ordem natural)
- **`tamanho`**: `P`, `M`, `G` (pode ser ordinal: P < M < G)
- **`material`**: `algodao`, `lã`, `sintetico` (nominal, **sem** ordem natural)

Cada linha é um produto. Não há valores numéricos originalmente — perfeito para estudar *encoding*.


In [ ]:

import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Criando o DataFrame didático
df = pd.DataFrame({
    'cor': ['azul', 'verde', 'vermelho', 'azul', 'vermelho', 'verde'],
    'tamanho': ['P', 'M', 'G', 'P', 'G', 'M'],
    'material': ['algodao', 'lã', 'sintetico', 'algodao', 'sintetico', 'lã']
})
display(df)
print('Formato:', df.shape)


,cor,tamanho,material
0,azul,P,algodao
1,verde,M,lã
2,vermelho,G,sintetico
3,azul,P,algodao
4,vermelho,G,sintetico
5,verde,M,lã


Formato: (6, 3)



---
## 1) One-Hot Encoding (**OHE**)
**O que é:** cria uma coluna binária (0/1) para **cada categoria** observada em cada coluna categórica.  
**Por que usar:** não impõe ordem fictícia e funciona bem com muitos modelos.  
**Cuidado:** pode **aumentar muito** o número de colunas se houver **muitas categorias únicas**.


In [ ]:

# One-Hot com scikit-learn (sem treinar modelo)
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_array = ohe.fit_transform(df[['cor','tamanho','material']])
ohe_cols = ohe.get_feature_names_out(['cor','tamanho','material'])

df_ohe = pd.DataFrame(ohe_array, columns=ohe_cols, index=df.index)
display(df_ohe)
print('Colunas geradas (OHE):')
print(list(df_ohe.columns))


,cor_azul,cor_verde,cor_vermelho,tamanho_G,tamanho_M,tamanho_P,material_algodao,material_lã,material_sintetico
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


Colunas geradas (OHE):
['cor_azul', 'cor_verde', 'cor_vermelho', 'tamanho_G', 'tamanho_M', 'tamanho_P', 'material_algodao', 'material_lã', 'material_sintetico']



**Como interpretar o resultado:**  
- Se `cor=azul`, a linha terá `cor_azul=1` e `cor_verde=0`, `cor_vermelho=0`.  
- Para `tamanho`, surgem colunas `tamanho_G`, `tamanho_M`, `tamanho_P`; a linha ativa apenas **uma** delas.  
- Para `material`, surgem `material_algodao`, `material_lã`, `material_sintetico`.

**Pergunta‑cheque (responda mentalmente):**  
1. Se uma linha tiver `cor=vermelho`, quais colunas de `cor_...` ficam em 1 e 0?



---
## 2) Ordinal Encoding (**OE**)
**O que é:** mapeia cada categoria para um **inteiro** (0,1,2, …).  
**Quando usar:** quando existe **ordem natural** entre categorias (ex.: P < M < G).  
**Risco:** em variáveis **nominais** (sem ordem), cria **ordem artificial** que pode induzir vieses.


In [ ]:

# Ordinal simples (apenas exemplo). Aqui NÃO definimos uma ordem semântica manualmente.
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
oe_array = oe.fit_transform(df[['cor','tamanho','material']])

df_oe = pd.DataFrame(oe_array, columns=['cor_ord','tamanho_ord','material_ord'], index=df.index)
display(df_oe)

print('Categorias aprendidas pelo OE (na ordem que o encoder viu no fit):')
for col, cats in zip(['cor','tamanho','material'], oe.categories_):
    print(f'{col}: {list(cats)} -> índices 0..{len(cats)-1}')


,cor_ord,tamanho_ord,material_ord
0,0.0,2.0,0.0
1,1.0,1.0,1.0
2,2.0,0.0,2.0
3,0.0,2.0,0.0
4,2.0,0.0,2.0
5,1.0,1.0,1.0


Categorias aprendidas pelo OE (na ordem que o encoder viu no fit):
cor: ['azul', 'verde', 'vermelho'] -> índices 0..2
tamanho: ['G', 'M', 'P'] -> índices 0..2
material: ['algodao', 'lã', 'sintetico'] -> índices 0..2



**Como interpretar o resultado:**  
- `tamanho_ord` pode ser lido como um **ranqueamento** (ex.: P=0, M=1, G=2).  
- Já para `cor` e `material` **não há ordem natural**. A codificação numérica pode levar modelos a "achar" que `vermelho > verde > azul`.

**Pergunta‑cheque:**  
2. Em qual(is) coluna(s) deste dataset o **Ordinal** faz sentido, e por quê?


**Resposta:** apenas a coluna Tamanho faz sentido porque há uma ordem natural entre as categorias de p,m,g.


---
## 3) Frequency Encoding (**FE**) — manual, passo a passo
**O que é:** substitui cada categoria pela sua **frequência relativa** na coluna.  
**Por que usar:** simples, funciona bem com **muitas categorias**.  
**Cuidado:** se você calcular a frequência usando **todo o dataset** e avaliar no mesmo, pode ocorrer **data leakage**.


In [ ]:

# Implementação manual SEM funções
df_freq = df.copy()

for c in df_freq.columns:
    freq_map = df_freq[c].value_counts(normalize=True)  # frequência por categoria (0..1)
    df_freq[c] = df_freq[c].map(freq_map).fillna(0.0)

display(df_freq)

print('Interpretação do exemplo:')
for c in df.columns:
    freq_map = df[c].value_counts(normalize=True)
    print(f'Coluna {c} -> frequências: {freq_map.to_dict()}')


,cor,tamanho,material
0,0.333333,0.333333,0.333333
1,0.333333,0.333333,0.333333
2,0.333333,0.333333,0.333333
3,0.333333,0.333333,0.333333
4,0.333333,0.333333,0.333333
5,0.333333,0.333333,0.333333


Interpretação do exemplo:
Coluna cor -> frequências: {'azul': 0.3333333333333333, 'verde': 0.3333333333333333, 'vermelho': 0.3333333333333333}
Coluna tamanho -> frequências: {'P': 0.3333333333333333, 'M': 0.3333333333333333, 'G': 0.3333333333333333}
Coluna material -> frequências: {'algodao': 0.3333333333333333, 'lã': 0.3333333333333333, 'sintetico': 0.3333333333333333}



**Como interpretar o resultado:**  
- Se `azul` aparece em 2/6 linhas, `cor=azul` vira `0.3333`.  
- Itens **raros** recebem valores menores; **comuns**, valores maiores.

**Pergunta‑cheque:**  
3. Se `material=algodao` aparece em 2 de 6 linhas, qual valor numérico você espera após o FE?


**Resposta:** esperaro o valor numérico 0.3333 para as linhas onde material é algodao


---
## 4) Target Mean Encoding (**TME**) — manual e didático
**O que é:** substitui a categoria pela **média do alvo** (\(\bar y\)) observada para aquela categoria.  
**Por que usar:** injeta informação **direta** da relação categoria → alvo.  
**Risco crítico:** **overfitting**/**leakage** se a média for calculada com dados que o modelo não deveria "ver". Em prática, faz‑se por *fold* no treino, com *smoothing*.


In [ ]:

# Vamos criar um alvo didático y (0/1) apenas para ilustrar o cálculo do TME
y = pd.Series([0, 1, 1, 0, 1, 0], name='y')

df_tmean = df.copy()
for c in df_tmean.columns:
    means = y.groupby(df_tmean[c]).mean()    # média de y por categoria dessa coluna
    df_tmean[c] = df_tmean[c].map(means).fillna(y.mean())

display(pd.concat([df, y, df_tmean.add_prefix('tmean_')], axis=1))


,cor,tamanho,material,y,tmean_cor,tmean_tamanho,tmean_material
0,azul,P,algodao,0,0.0,0.0,0.0
1,verde,M,lã,1,0.5,0.5,0.5
2,vermelho,G,sintetico,1,1.0,1.0,1.0
3,azul,P,algodao,0,0.0,0.0,0.0
4,vermelho,G,sintetico,1,1.0,1.0,1.0
5,verde,M,lã,0,0.5,0.5,0.5



**Como interpretar o resultado:**  
- Se, no nosso `y` didático, itens `material=algodao` têm média 0.5 e `material=lã` têm 1.0, então `tmean_material` vira 0.5 ou 1.0 conforme a categoria da linha.  
- Esses valores **já** carregam sinal do alvo — ótimo para alguns modelos, mas **perigoso** se calculado de forma ingênua.

**Pergunta‑cheque:**  
4. Por que o TME é especialmente suscetível a *data leakage*? Qual seria uma prática correta para evitá‑lo?


**Resposta:** O Target Mean Encoding é suscetível a data leakage porque ele usa a média do alvo (y) para substituir as categorias. Se você calcular essa média usando todo o dataset (conjuntos de treino e teste juntos), o modelo terá acesso a informações do conjunto de teste durante a fase de treinamento através dos valores codificados. Isso faz com que o modelo pareça ter um desempenho melhor do que realmente teria em dados não vistos.

Uma prática correta para evitar o data leakage com TME seria:
Dividir os dados em conjuntos de treino e teste antes de aplicar o TME.


---
## 5) Misturando encoders por coluna com `ColumnTransformer` (somente transformação)
Ex.: aplicar **One-Hot** em `cor` e `material` (nominais) e **Ordinal** em `tamanho` (ordinal).


In [ ]:

cols_ohe = ['cor', 'material']
cols_ord = ['tamanho']

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cols_ohe),
    ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cols_ord)
], remainder='drop')

arr = ct.fit_transform(df)
col_names = list(ct.named_transformers_['ohe'].get_feature_names_out(cols_ohe)) + ['tamanho_ord']

df_mix = pd.DataFrame(arr, columns=col_names, index=df.index)
display(df_mix)
print('Colunas finais (mix OHE + OE):', list(df_mix.columns))


,cor_azul,cor_verde,cor_vermelho,material_algodao,material_lã,material_sintetico,tamanho_ord
0,1.0,0.0,0.0,1.0,0.0,0.0,2.0
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,2.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,0.0,1.0,0.0,0.0,1.0,0.0,1.0


Colunas finais (mix OHE + OE): ['cor_azul', 'cor_verde', 'cor_vermelho', 'material_algodao', 'material_lã', 'material_sintetico', 'tamanho_ord']



---
## ✅ Recap — quando usar qual encoding?
- **One-Hot**: padrão seguro para variáveis **nominais**; atenção à **dimensionalidade**.  
- **Ordinal**: apenas quando há **ordem natural** comprovada (ex.: P < M < G).  
- **Frequência**: útil em **alta cardinalidade**; cuidado com *leakage*.  
- **Target Mean**: poderoso, mas **suscetível a leakage**; use **dentro do treino por fold** e com *smoothing*.

> **Dica prática**: em projetos reais, integre *encoders* a **Pipelines** (com validação correta) antes de avaliar modelos.



---
## 📝 Mini‑exercícios (responda no próprio notebook)

1. **Explique** com suas palavras por que o One‑Hot **não** impõe ordem entre categorias.  
2. No nosso dataset, substitua manualmente a ordem de `tamanho` para refletir `P < M < G` **antes** de aplicar o `OrdinalEncoder`. Mostre o resultado.  
3. Recrie o **Frequency Encoding** **apenas** para a coluna `cor` e explique o que muda se adicionarmos mais linhas com `vermelho`.  
4. No TME, imagine que `y` é altamente desequilibrado. **O que aconteceria** com as médias por categoria? Dê um exemplo com duas categorias e proponha uma **mitigação**.
5. Analise abaixo o dataset Mushroom do Kaggle (https://www.kaggle.com/datasets/uciml/mushroom-classification) aplicando o conceito de Encoding.

> Dica: adicione células abaixo desta seção para suas respostas/códigos.


1.  não impõe uma ordem entre as categorias porque ele transforma cada categoria em uma coluna binária separada o One-Hot cria uma nova coluna para cada valor único da variável original

2.

In [ ]:
tamanho_order = ['P', 'M', 'G']

oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
oe.fit(df[['cor','tamanho','material']])

oe_ordered = OrdinalEncoder(categories=[oe.categories_[0], tamanho_order, oe.categories_[2]],
                            handle_unknown='use_encoded_value',
                            unknown_value=-1)

oe_ordered_array = oe_ordered.fit_transform(df[['cor','tamanho','material']])

df_oe_ordered = pd.DataFrame(oe_ordered_array, columns=['cor_ord','tamanho_ord','material_ord'], index=df.index)
display(df_oe_ordered)

print('Categorias aprendidas pelo OE (com ordem especificada para tamanho):')
for col, cats in zip(['cor','tamanho','material'], oe_ordered.categories_):
    print(f'{col}: {list(cats)} -> índices 0..{len(cats)-1}')

,cor_ord,tamanho_ord,material_ord
0,0.0,0.0,0.0
1,1.0,1.0,1.0
2,2.0,2.0,2.0
3,0.0,0.0,0.0
4,2.0,2.0,2.0
5,1.0,1.0,1.0


Categorias aprendidas pelo OE (com ordem especificada para tamanho):
cor: ['azul', 'verde', 'vermelho'] -> índices 0..2
tamanho: ['P', 'M', 'G'] -> índices 0..2
material: ['algodao', 'lã', 'sintetico'] -> índices 0..2


# 3.

In [ ]:
# Recriando o Frequency Encoding apenas para a coluna 'cor'
df_freq_cor = df.copy()

freq_map_cor = df_freq_cor['cor'].value_counts(normalize=True)  # frequência por categoria (0..1)
df_freq_cor['cor_freq'] = df_freq_cor['cor'].map(freq_map_cor).fillna(0.0)

display(df_freq_cor[['cor', 'cor_freq']])

print('Frequências originais para a coluna "cor":')
print(freq_map_cor.to_dict())

**Explicação sobre a adição de mais linhas com 'vermelho':**

Se adicionarmos mais linhas com a cor 'vermelho' ao dataset **original** e então aplicarmos o Frequency Encoding na coluna 'cor' **novamente**, as frequências relativas mudarão.

Especificamente a frequência da categoria vermelho aumentará e as frequências das categorias azul e verde diminuirão, pois o número total de linhas no dataset aumentou, mas a contagem de 'azul' e 'verde' permaneceu a mesma.

Isso demonstra como o Frequency Encoding reflete a distribuição das categorias nos dados. Categorias que aparecem com mais frequência receberão valores codificados maiores, e categorias menos frequentes receberão valores menores.

# 4.

Quando o alvo (y) é muito desequilibrado, o TME para categorias com poucas ocorrências pode ser muito influenciado por essas poucas observações, tornando o valor codificado menos confiável e podendo levar a overfitting.

Para mitigar: Usar o smoothing (suavização). O smoothing mistura a média da categoria com a média global do alvo, dando mais peso à média global para categorias com poucas amostras. Isso torna os valores codificados mais estáveis e menos suscetíveis a flutuações em categorias raras, especialmente quando o alvo é desequilibrado.